In [2]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk
import mysql.connector
import ssl

# konksi nanti coba ditaruh diluar
# koneksi
db = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='UnNgGrape'
    )
cursor = db.cursor()

class Scrape:

    def scrapeMain(keyword):
        # inisialisasi array untuk menampung hasil data
        asal_situs = []
        title_lowongan = []
        nama_perusahaan = []
        lokasi_perusahaan = []
        keterangan_lowongan = []
        skill_lowongan = []
        benefit_lowongan = []
        deskripsi_lowongan = []
        stem_detail = []
        link_lowongan = []

        # pake try except (continue) kalau udah looping keyword
        # loop scrape
        cursor.execute("SELECT * FROM `scrape` WHERE `asal_situs` LIKE 'glints'")
        raw_scrape = cursor.fetchall()
        for rowS in raw_scrape:
            asal = rowS[0]
            # scrape data
            main_link1 = rowS[1]
            main_link2 = rowS[2]
            main_link = main_link1 + keyword + main_link2
            tag_main = rowS[3]
            tag_lowongan = rowS[4]
            tag_lowongan_part = rowS[5]
            tag_perusahaan = rowS[6]
            tag_perusahaan_part = rowS[7]
            tag_lokasi = rowS[8]
            tag_lokasi_part = rowS[9]
            tag_mainDetail = rowS[10]
            tag_keterangan = rowS[11]
            tag_skill = rowS[12]
            tag_benefit = rowS[13]
            tag_deskripsi = rowS[14]
            breakDeskripsi = rowS[15]
            raw_link = rowS[16]

            # tab sekali jika sudah masuk looping scrape
            r = Request(main_link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
            response = urlopen(r).read()
            soup = BeautifulSoup(response, "lxml")
            print(type(soup))

            jobList = soup.find_all("div", tag_main)
            for p in jobList:
                # link scrape dulu untuk cek data sudah pernah di scrape atau belum
                link = raw_link+p.find('a').get('href')
                lowongan = p.find(tag_lowongan_part, tag_lowongan).get_text()
                # menggunakan try except karena ada beberapa perusahaan yang dirahasiakan
                try:
                    perusahaan = p.find(tag_perusahaan_part, tag_perusahaan).get_text()
                except:
                    perusahaan = "Perusahaan Dirahasiakan"

                try:
                    lokasi = p.find(tag_lokasi_part, tag_lokasi).get_text().replace("head office - ", "")
                except:
                    lokasi = "-"

                # try print return scrapeDetail
                keterangan, skill, benefit, deskripsi = Scrape.scrapeDetail(link, tag_mainDetail, tag_keterangan, tag_skill, tag_benefit, tag_deskripsi, breakDeskripsi)

                print(Scrape.scrapeDetail(link, tag_mainDetail, tag_keterangan, tag_skill, tag_benefit, tag_deskripsi, breakDeskripsi))
                print("\n")

                # stemming for detail
                raw_stem = lowongan + " " + perusahaan + " " + lokasi + " " + str(keterangan) + " " + str(skill) + " " + str(benefit) + " " + str(deskripsi)
                stem = Scrape.stemming(raw_stem)

                # push to array penampung hasil data
                asal_situs.append(asal)
                title_lowongan.append(lowongan)
                nama_perusahaan.append(perusahaan)
                lokasi_perusahaan.append(lokasi)
                keterangan_lowongan.append(keterangan)
                skill_lowongan.append(skill)
                benefit_lowongan.append(benefit)
                deskripsi_lowongan.append(deskripsi)
                stem_detail.append(stem)
                link_lowongan.append(link)

        # tampil pakai panda
        jobList_dict ={'asal':asal_situs, 'lowongan':title_lowongan, 'perusahaan':nama_perusahaan, 'lokasi':lokasi_perusahaan, 'keterangan':keterangan_lowongan, 'skill':skill_lowongan, 'benefit':benefit_lowongan, 'deskripsi':deskripsi_lowongan, 'stem':stem_detail, 'link':link_lowongan}
        df = pd.DataFrame(jobList_dict,columns = ['asal', 'lowongan', 'perusahaan', 'lokasi', 'keterangan', 'skill', 'benefit', 'deskripsi', 'stem', 'link'])
        return df.sort_values('asal',ascending=True)

    def scrapeDetail(linkDetail, tag_mainDetail, tag_keterangan, tag_skill, tag_benefit, tag_deskripsi, breakDeskripsi):
        r = Request(linkDetail, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
        response = urlopen(r).read()
        soup = BeautifulSoup(response, "lxml")

        raw_detail = soup.find_all("div", tag_mainDetail)
        for p in raw_detail:
            # menggunakan try except karena ada beberapa estimasi keterangan yang tidak terlampir
            try:
                raw_keterangan = p.find("div", tag_keterangan ).get_text(separator=". ").replace("Fungsi Kerja. ", "")
                nltk_tokens = nltk.sent_tokenize(raw_keterangan)
                keterangan = ""

                for x in nltk_tokens:
                    if x == "Lamar.":
                        break
                    keterangan = keterangan + " " + x
            except:
                keterangan = "-"
            
            try:
                skill = p.find("div", tag_skill).get_text(separator=" ")
            except:
                skill = "-"
            try:
                benefit = p.find("div", tag_benefit).get_text(separator=" ").replace("Tunjangan dan keuntungan", "keuntungan:")
            except:
                benefit = "-"

            # menggunakan try except karena ada beberapa deskripsi yang NoneType
            try:
                # replace untuk glints
                raw_deskripsi = p.find("div", tag_deskripsi ).get_text(separator=". ").replace("Informasi Penting. Pastikan perusahaan yang kamu lamar resmi dengan memeriksa website dan lowongan kerja mereka.. Read Less.", "")
                nltk_tokens = nltk.sent_tokenize(raw_deskripsi)
                deskripsi = ""

                for x in nltk_tokens:
                    if x == breakDeskripsi:
                        break
                    deskripsi = deskripsi + " " + x
            except:
                deskripsi = "-"
            
            # give to scrapeMain
            return keterangan, skill, benefit, deskripsi

    def stemming(raw_stem):
        # define punctuation
        punctuations = '''!()-[]{};:=+`'",<>./|\?@#$%^&*_~'''

        # remove punctuation from the string
        no_punct = ""
        for char in raw_stem:
            if char not in punctuations:
                no_punct = no_punct + char
            else:
                no_punct = no_punct + " "

        # display the unpunctuated string
        raw_stem = no_punct.lower()

        # create stemmer
        # nltk
        ps = PorterStemmer()
        # sastrawi
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()

        # stemming process
        # nltk
        words = word_tokenize(raw_stem)
        nltk_stemmer = ""
        for w in words:
            nltk_stemmer = nltk_stemmer + " " + ps.stem(w)
        # sastrawi sekaligus return
        return stemmer.stem(nltk_stemmer)

# print(Scrape.getKeyword())
# print("\n")

Scrape.scrapeMain("palembang")

<class 'bs4.BeautifulSoup'>
(' Operations. Full-Time. 1 - 3 tahun pengalaman.', 'Skills wajib Mechanical Cycle Teamwork Like Sporty Good Communication Skills Skills tambahan Like To Challenge Good Interpersonal Skills Good Attitude Good Skill For Bycyle', 'keuntungan: Lainnya', '  Deskripsi pekerjaan Mekanik Rodalink Palembang Polygon Bikes. MEKANIK RODALINK PALEMBANG. Tanggung Jawab. Melakukan perakitan sepeda.. Memastikan seluruh kondisi sepeda layak dan siap pakai.. Memberikan jasa layanan servis dan perbaikan.. Mengelola dan menjaga kelengkapan peralatan kerja.. Menjalin dan menjaga hubungan baik dengan customer.. Persyaratan. Pendidikan min. SMA/SMK atau sederajat.. Pengalaman kerja min. 1 tahun di bidang perbengkelan (terbuka untuk fresh graduates).. Menguasai produk dan suku cadang.. Menguasai sistem mekanika sepeda.. Menyukai olah raga (utamanya bersepeda). Bersedia bekerja dengan sistem shift. Penempatan Palembang')


(' IDR. 0. - . 3,400,000. /. month. Operations. Freelance. 

,asal,lowongan,perusahaan,lokasi,keterangan,skill,benefit,deskripsi,stem,link
0,glints,Mekanik Rodalink Palembang,Polygon Bikes,Palembang,Operations. Full-Time. 1 - 3 tahun pengalaman.,Skills wajib Mechanical Cycle Teamwork Like Sp...,keuntungan: Lainnya,Deskripsi pekerjaan Mekanik Rodalink Palemba...,mekanik rodalink palembang polygon bike palemb...,https://glints.com/id/opportunities/jobs/mekan...
27,glints,Marketing Communication Staff,Act Foundation Regional Sumatra Bagian Selatan,Palembang,Marketing. Full-Time. 1 - 3 tahun pengalaman.,Skills wajib Target Oriented Marketing Communi...,-,Deskripsi pekerjaan Marketing Communication ...,market commun staff act foundat region sumatra...,https://glints.com/id/opportunities/jobs/marke...
26,glints,Partnership Staff,Act Foundation Regional Sumatra Bagian Selatan,Palembang,Marketing. Full-Time. 1 - 3 tahun pengalaman.,Skills wajib Target Oriented Negotiation Skill...,-,Deskripsi pekerjaan Partnership Staff Act Fo...,partnership staff act foundat region sumatra b...,https://glints.com/id/opportunities/jobs/partn...
25,glints,Partnership Staff,Act Foundation Regional Sumatra Bagian Selatan,Palembang,Marketing. Full-Time. 1 - 3 tahun pengalaman.,Skills wajib Target Oriented Negotiation Skill...,-,Deskripsi pekerjaan Partnership Staff Act Fo...,partnership staff act foundat region sumatra b...,https://glints.com/id/opportunities/jobs/partn...
24,glints,Direct Sales Supervisor,PT Eka Mas Republik,"Tebet, Jakarta",Business Development / Sales. Full-Time. 1 - ...,Skills wajib Direct Selling Team Leadership Do...,keuntungan: Asuransi kesehatan,Deskripsi pekerjaan Direct Sales Supervisor ...,direct sale supervisor pt eka ma republik tebe...,https://glints.com/id/opportunities/jobs/direc...
23,glints,Merchant Onboarding Manager,Raena,Jakarta,Business Development / Sales. Full-Time. 3 - ...,Skills wajib Beauty Industry Sales Account Man...,-,Deskripsi pekerjaan Merchant Onboarding Mana...,merchant onboard manag raena jakarta busi deve...,https://glints.com/id/opportunities/jobs/merch...
22,glints,Admin After Sales Service,PT Gree Electric Appliances Indonesia,Palembang,Administrative. Full-Time. 1 - 3 tahun pengal...,Skills wajib Communication Skills Microsoft Of...,-,Deskripsi pekerjaan Admin After Sales Servic...,admin after sale servic pt gree electr applian...,https://glints.com/id/opportunities/jobs/admin...
21,glints,Store Supervisor - Lampung,Pt. Dom Pizza Indonesia,Bandar Lampung,"IDR. 4,000,000. - . 4,300,000. /. month. Oper...",Skills wajib Teamwork Leadership PnL Management,-,Deskripsi pekerjaan Store Supervisor - Lampu...,store supervisor lampung pt dom pizza indonesi...,https://glints.com/id/opportunities/jobs/store...
20,glints,IT Support Streaming,Bali United Football Club,Jakarta,Engineering. Full-Time. 3 - 5 tahun pengalaman.,Skills wajib UDP/TCP Microtic Router Troublesh...,-,Deskripsi pekerjaan IT Support Streaming Bal...,it support stream bal unit footbal club jakart...,https://glints.com/id/opportunities/jobs/it-su...
19,glints,Mandarin Executive,PT Gree Electric Appliances Indonesia,"Penjaringan, Jakarta",Operations. Full-Time. Pengalaman kurang dari...,Skills wajib Mandarin Communication Easily Ada...,-,Deskripsi pekerjaan Mandarin Executive PT Gr...,mandarin execut pt gree electr applianc indone...,https://glints.com/id/opportunities/jobs/manda...
